### Add audio results to output.db

In [ ]:
# read pkl file
quantex_results = pd.read_pickle("/home/nele_pauline_suffo/outputs/vtc/quantex_df.pkl")
# remove _16khz from audio_file_name 
quantex_results['audio_file_name'] = quantex_results['audio_file_name'].str.replace('_16kHz', '', regex=False)

#quantex_results = df[0:10]

# Load frame-wise detection results and video info from the database
db_path = '/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db'
with sqlite3.connect(db_path) as conn:
    frame_df = pd.read_sql_query("SELECT * FROM Detections", conn)
    videos_info_df = pd.read_sql_query("SELECT video_id, video_path FROM Videos", conn)

# Merge video_id into quantex_results
quantex_results = pd.merge(quantex_results, videos_info_df[['video_id', 'video_path']], left_on='audio_file_name', right_on='video_path', how='left')

# Map RTTM annotations to frames
fps = 30  # Assuming a frame rate of 30 FPS

# Initialize new columns for speaker types in frame_df
speaker_types = ['KCHI', 'FEM', 'MAL', 'CHI']  # Define target speaker type columns
for speaker_col in speaker_types:
    frame_df[speaker_col] = 0

# Assign speaker annotations to frames for KCHI, CDS, OHS
if not quantex_results.empty:  # Proceed only if quantex_results has data after merge
    for _, rttm_row in quantex_results.iterrows():
        rttm_video_id = rttm_row['video_id']
        speaker_label_from_rttm = rttm_row['Voice_type']  # Label from RTTM 'Speaker' column
        utterance_start_time = rttm_row['Utterance_Start']
        utterance_end_time = rttm_row['Utterance_End']

        # Determine which speaker type column to update
        if speaker_label_from_rttm not in speaker_types:
            continue  # Skip unknown speaker types

        # Convert time to frame numbers (inclusive)
        start_frame = int(utterance_start_time * fps)
        end_frame = int(utterance_end_time * fps)

        #print(f"Convert {utterance_start_time} - {utterance_end_time} for video {rttm_video_id} with speaker type {speaker_label_from_rttm} to {start_frame} - {end_frame}")

        # Set the speaker label to 1 for all frames in range
        frame_mask = (
            (frame_df['video_id'] == rttm_video_id) &
            (frame_df['frame_number'] >= start_frame) &
            (frame_df['frame_number'] <= end_frame)
        )
        frame_df.loc[frame_mask, speaker_label_from_rttm] = 1

# To view the first few rows of the modified frame_df:
# print("Modified frame_df head:")
# print(frame_df.head())

# To view rows where speaker types are active:
# print("\nFrames with speaker activity:")
# print(frame_df[(frame_df['KCHI'] == 1) | (frame_df['CDS'] == 1) | (frame_df['OHS'] == 1)].head())

#Save the updated frame-wise detection results back to the database (optional)
db_path = '/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db'
with sqlite3.connect(db_path) as conn:
    # Save the new table
    frame_df.to_sql('Detections_with_speaker', conn, if_exists='replace', index=False)

    # List all tables to confirm it's there
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
    print(tables)

## 1. How many utterances does the key child produce?


In [6]:
import sqlite3
import pandas as pd
import os

# show all columns in pandas DataFrame
pd.set_option('display.max_columns', None)

In [30]:

query_01 = """
WITH RandomSubjects AS (
    SELECT DISTINCT s.child_id, s.video_name, v.video_id, s.age_at_recording
    FROM Subjects s
    JOIN Videos v ON s.video_name = v.video_path
    ORDER BY RANDOM()
    LIMIT 40
),
-- Ziehe alle möglichen Frames aus den Videos der RandomSubjects
AllFrames AS (
    SELECT 
        rs.child_id, 
        rs.video_id, 
        rs.age_at_recording,
        f.frame_number
    FROM RandomSubjects rs
    JOIN (
        -- Hier alle möglichen Frames pro Video bestimmen
        SELECT video_id, frame_number
        FROM Detections_with_speaker
        GROUP BY video_id, frame_number
    ) f ON rs.video_id = f.video_id
),
-- Ziehe zufällig 5000 Frames
RandomFrames AS (
    SELECT *
    FROM AllFrames
    ORDER BY RANDOM()
    LIMIT 5000
)
SELECT 
    rf.video_id,
    rf.frame_number,
    rf.child_id,
    rf.age_at_recording AS age,
    d.proximity,
    CASE
        WHEN CAST(d.object_class AS INTEGER) >= 5 AND CAST(d.object_class AS INTEGER) <= 10 THEN d.object_class
        ELSE 'none'
    END AS object_class,
    COALESCE(d.gaze_direction, 'none') AS gaze_direction,
    COALESCE(d.KCHI, 0) AS kchi_present,
    COALESCE(CASE WHEN d.FEM = 1 OR d.CHI = 1 OR d.MAL = 1 THEN 1 ELSE 0 END, 0) AS cds_present,
    COALESCE(CASE WHEN CAST(d.object_class AS INTEGER) = 2 OR CAST(d.object_class AS INTEGER) = 3 THEN 1 ELSE 0 END, 0) AS face_present,
    COALESCE(CASE WHEN CAST(d.object_class AS INTEGER) = 0 THEN 1 ELSE 0 END, 0) AS child_present,
    COALESCE(CASE WHEN CAST(d.object_class AS INTEGER) = 1 THEN 1 ELSE 0 END, 0) AS adult_present,
    CASE 
        WHEN 
            COALESCE(d.FEM, 0) = 1 OR COALESCE(d.CHI, 0) = 1 OR COALESCE(d.MAL, 0) = 1
            OR COALESCE(d.proximity, 0) > 0.5
            OR COALESCE(CAST(d.object_class AS INTEGER), -1) = 0
            OR COALESCE(CAST(d.object_class AS INTEGER), -1) = 1
            THEN 'social'
        ELSE 'alone'
    END AS play_context,
    CASE 
        WHEN COALESCE(CAST(d.object_class AS INTEGER), -1) = 1 THEN 'adult'
        WHEN COALESCE(CAST(d.object_class AS INTEGER), -1) = 0 THEN 'child'
        ELSE 'none'
    END AS person_age_class
FROM RandomFrames rf
LEFT JOIN Detections_with_speaker d
    ON rf.video_id = d.video_id
   AND rf.frame_number = d.frame_number
ORDER BY rf.child_id, rf.video_id, rf.frame_number;
"""

def merge_duplicates_01(df):
    # Mapping for object_class
    object_class_map = {
        '5': 'book',
        '6': 'toy',
        '7': 'kitchenware',
        '8': 'screen',
        '9': 'food',
        '10': 'other object'
    }

    grouped = df.groupby(['video_id', 'frame_number', 'child_id', 'age'])
    merged_rows = []

    for name, group in grouped:
        merged_row = {
            'video_id': name[0],
            'frame_number': name[1],
            'child_id': name[2],
            'age': name[3],
            'kchi_present': int(group['kchi_present'].any()),
            'cds_present': int(group['cds_present'].any()),
            'face_present': int(group['face_present'].any()),
            'child_present': int(group['child_present'].any()),
            'adult_present': int(group['adult_present'].any()),
            'play_context': (
                'social' if 'social' in group['play_context'].values
                else 'alone' if all(pc == 'alone' for pc in group['play_context'].values)
                else 'none'
            )
        }

        # --- object_class: only one column, mapping, 'multiple' if >1 ---
        raw_object_classes = [str(v) for v in group['object_class'].unique() if v not in ('none', 'NaN')]
        mapped_object_classes = [object_class_map.get(oc, oc) for oc in raw_object_classes]
        mapped_object_classes = [oc for oc in mapped_object_classes if oc not in ('none', 'NaN')]
        if len(mapped_object_classes) == 1:
            merged_row['object_class'] = mapped_object_classes[0]
        elif len(mapped_object_classes) > 1:
            merged_row['object_class'] = 'multiple'
        else:
            merged_row['object_class'] = 'none'

        # --- person_age_class logic ---
        pac_set = set(v for v in group['person_age_class'].unique() if v not in ('none', 'NaN'))
        if pac_set == {'child'}:
            merged_row['person_age_class'] = 'child'
        elif pac_set == {'adult'}:
            merged_row['person_age_class'] = 'adult'
        elif pac_set == {'child', 'adult'}:
            merged_row['person_age_class'] = 'both'
        else:
            merged_row['person_age_class'] = 'none'

        # --- proximity and gaze_direction: take max proximity and corresponding gaze ---
        proximities = pd.to_numeric(group['proximity'], errors='coerce').fillna(-1)
        if not proximities.empty and proximities.max() >= 0:
            max_prox_idx = proximities.idxmax()
            max_prox_value = proximities.loc[max_prox_idx]
            merged_row['proximity'] = max_prox_value
            gaze_value = group.loc[max_prox_idx, 'gaze_direction']
            if gaze_value in ('none', 'NaN', None):
                gaze_value = 'none'
            merged_row['gaze_direction'] = gaze_value
        else:
            merged_row['proximity'] = None
            merged_row['gaze_direction'] = 'none'

        merged_rows.append(merged_row)

    return pd.DataFrame(merged_rows)


In [ ]:
# Load data
with sqlite3.connect('/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db') as conn:
    utterance_01_df = pd.read_sql_query(query_01, conn)

utterance_01_df_merged = merge_duplicates_01(utterance_01_df)
utterance_01_df_merged.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/utterance_01_df.csv', index=False)

utterance_01_df_merged.head(10)

,video_id,frame_number,child_id,age,kchi_present,cds_present,face_present,child_present,adult_present,play_context,object_class,person_age_class,proximity,gaze_direction
0,11,100,263284,3.8,0,0,0,0,0,alone,multiple,none,NaN,none
1,11,490,263284,3.8,0,0,0,0,0,alone,none,none,NaN,none
2,11,1190,263284,3.8,0,0,1,0,1,social,none,adult,0.290025,1.0
3,11,1200,263284,3.8,0,0,1,0,1,social,other object,adult,0.301284,1.0
4,11,1370,263284,3.8,0,0,0,0,0,alone,other object,none,NaN,none
5,11,1460,263284,3.8,0,0,0,0,0,alone,none,none,NaN,none
6,11,1550,263284,3.8,0,0,0,0,0,alone,none,none,NaN,none
7,11,2140,263284,3.8,1,0,1,0,1,social,none,adult,1.000000,1.0
8,11,2150,263284,3.8,1,0,0,0,1,social,none,adult,NaN,none
9,11,2270,263284,3.8,0,0,0,0,1,social,none,adult,NaN,none


#### a) alone vs. social: kchi_present ~ age * context + (context|child_id) with context being a factor with levels „social“ and „alone“

In [32]:
utterance_01a_df = utterance_01_df_merged[['video_id', 'frame_number', 'child_id', 'kchi_present', 'age', 'play_context']]
utterance_01a_df.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/utterance_01a_df.csv', index=False)
len(utterance_01a_df), utterance_01a_df.head(10)

(5000,
    video_id  frame_number  child_id  kchi_present  age play_context
 0        11           100    263284             0  3.8        alone
 1        11           490    263284             0  3.8        alone
 2        11          1190    263284             0  3.8       social
 3        11          1200    263284             0  3.8       social
 4        11          1370    263284             0  3.8        alone
 5        11          1460    263284             0  3.8        alone
 6        11          1550    263284             0  3.8        alone
 7        11          2140    263284             1  3.8       social
 8        11          2150    263284             1  3.8       social
 9        11          2270    263284             0  3.8       social)

#### b) only alone data: kchi_present ~ age*object + (object | child_id) see if predictor object makes a difference, with object being a factor with levels „toy“, „book“, ...

In [34]:
utterance_01b_df = utterance_01_df_merged[utterance_01_df_merged['play_context'] == 'alone']
utterance_01b_df = utterance_01b_df[['video_id', 'frame_number', 'child_id', 'kchi_present', 'age', 'object_class']]
utterance_01b_df.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/utterance_01b_df.csv', index=False)
len(utterance_01b_df), utterance_01b_df.head(10)

(2197,
     video_id  frame_number  child_id  kchi_present  age  object_class
 0         11           100    263284             0  3.8      multiple
 1         11           490    263284             0  3.8          none
 4         11          1370    263284             0  3.8  other object
 5         11          1460    263284             0  3.8          none
 6         11          1550    263284             0  3.8          none
 11        11          3030    263284             0  3.8      multiple
 13        11          3080    263284             0  3.8           toy
 14        11          4110    263284             0  3.8          none
 16        11          4500    263284             0  3.8          none
 17        11          5120    263284             0  3.8           toy)

#### c) only social data: kchi_present ~ age * age_class * face * gaze + (age_class * face * gaze|child_id) see if type of social interaction makes a differences, that is the age class of the other person, whether there is a face and whether there is gaze

In [35]:
utterance_01c_df = utterance_01_df_merged[utterance_01_df_merged['play_context'] == 'social']
utterance_01c_df = utterance_01c_df[['video_id', 'frame_number', 'child_id', 'kchi_present', 'age', 'face_present', 'person_age_class', 'gaze_direction']]
utterance_01c_df.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/utterance_01c_df.csv', index=False)
len(utterance_01c_df), utterance_01c_df.head(10)

(2803,
     video_id  frame_number  child_id  kchi_present  age  face_present  \
 2         11          1190    263284             0  3.8             1   
 3         11          1200    263284             0  3.8             1   
 7         11          2140    263284             1  3.8             1   
 8         11          2150    263284             1  3.8             0   
 9         11          2270    263284             0  3.8             0   
 10        11          2640    263284             1  3.8             0   
 12        11          3070    263284             0  3.8             1   
 15        11          4370    263284             0  3.8             0   
 18        11          5450    263284             1  3.8             1   
 20        11          7030    263284             1  3.8             0   
 
    person_age_class gaze_direction  
 2             adult            1.0  
 3             adult            1.0  
 7             adult            1.0  
 8             adult     

## 2. How much speech is directed at the key child?

In [7]:

query_02 = """
WITH RandomSubjects AS (
    SELECT DISTINCT s.child_id, s.video_name, v.video_id, s.age_at_recording
    FROM Subjects s
    JOIN Videos v ON s.video_name = v.video_path
    ORDER BY RANDOM()
    LIMIT 40
),
-- Ziehe alle möglichen Frames aus den Videos der RandomSubjects
AllFrames AS (
    SELECT 
        rs.child_id, 
        rs.video_id, 
        rs.age_at_recording,
        f.frame_number
    FROM RandomSubjects rs
    JOIN (
        -- Hier alle möglichen Frames pro Video bestimmen
        SELECT video_id, frame_number
        FROM Detections_with_speaker
        GROUP BY video_id, frame_number
    ) f ON rs.video_id = f.video_id
),
-- Ziehe zufällig 5000 Frames
RandomFrames AS (
    SELECT *
    FROM AllFrames
    ORDER BY RANDOM()
    LIMIT 5000
)
SELECT 
    rf.video_id,
    rf.frame_number,
    rf.child_id,
    rf.age_at_recording AS age,
    d.proximity,
    COALESCE(d.gaze_direction, 'none') AS gaze_direction,
    COALESCE(CASE WHEN d.FEM = 1 OR d.CHI = 1 OR d.MAL = 1 THEN 1 ELSE 0 END, 0) AS cds_present,
    COALESCE(CASE WHEN CAST(d.object_class AS INTEGER) = 2 OR CAST(d.object_class AS INTEGER) = 3 THEN 1 ELSE 0 END, 0) AS face_present,
    COALESCE(CASE WHEN CAST(d.object_class AS INTEGER) = 0 THEN 1 ELSE 0 END, 0) AS child_present,
    COALESCE(CASE WHEN CAST(d.object_class AS INTEGER) = 1 THEN 1 ELSE 0 END, 0) AS adult_present,
    CASE 
        WHEN COALESCE(CAST(d.object_class AS INTEGER), -1) = 1 THEN 'adult'
        WHEN COALESCE(CAST(d.object_class AS INTEGER), -1) = 0 THEN 'child'
        ELSE 'none'
    END AS person_age_class
FROM RandomFrames rf
LEFT JOIN Detections_with_speaker d
    ON rf.video_id = d.video_id
   AND rf.frame_number = d.frame_number
ORDER BY rf.child_id, rf.video_id, rf.frame_number;
"""

def merge_duplicates_02(df):
    grouped = df.groupby(['video_id', 'frame_number', 'child_id', 'age'])
    merged_rows = []

    for name, group in grouped:
        merged_row = {
            'video_id': name[0],
            'frame_number': name[1],
            'child_id': name[2],
            'age': name[3],
            'cds_present': int(group['cds_present'].any()),
            'face_present': int(group['face_present'].any()),
            'child_present': int(group['child_present'].any()),
            'adult_present': int(group['adult_present'].any()),
        }

        # --- person_age_class logic ---
        pac_set = set(v for v in group['person_age_class'].unique() if v not in ('none', 'NaN'))
        if pac_set == {'child'}:
            merged_row['person_age_class'] = 'child'
        elif pac_set == {'adult'}:
            merged_row['person_age_class'] = 'adult'
        elif pac_set == {'child', 'adult'}:
            merged_row['person_age_class'] = 'both'
        else:
            merged_row['person_age_class'] = 'none'

        # --- proximity and gaze_direction: take max proximity and corresponding gaze ---
        proximities = pd.to_numeric(group['proximity'], errors='coerce').fillna(-1)
        if not proximities.empty and proximities.max() >= 0:
            max_prox_idx = proximities.idxmax()
            max_prox_value = proximities.loc[max_prox_idx]
            merged_row['proximity'] = max_prox_value
            gaze_value = group.loc[max_prox_idx, 'gaze_direction']
            if gaze_value in ('none', 'NaN', None):
                gaze_value = 'none'
            merged_row['gaze_direction'] = gaze_value
        else:
            merged_row['proximity'] = None
            merged_row['gaze_direction'] = 'none'

        merged_rows.append(merged_row)

    return pd.DataFrame(merged_rows)


#### cds_present ~ age * age_class * face * gaze * proximity + (age_class * face * gaze|child_id)


In [8]:
# Load data
with sqlite3.connect('/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db') as conn:
    utterance_02_df = pd.read_sql_query(query_02, conn)

utterance_02_df_merged = merge_duplicates_02(utterance_02_df)
utterance_02_df_merged.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/utterance_02_df.csv', index=False)

utterance_02_df_merged.head(10)

,video_id,frame_number,child_id,age,cds_present,face_present,child_present,adult_present,person_age_class,proximity,gaze_direction
0,3,490,263190,3.73,1,0,0,0,none,NaN,none
1,3,840,263190,3.73,0,0,0,0,none,NaN,none
2,3,850,263190,3.73,0,0,0,0,none,NaN,none
3,3,1060,263190,3.73,0,0,0,0,none,NaN,none
4,3,1440,263190,3.73,0,0,0,0,none,NaN,none
5,3,1710,263190,3.73,0,0,0,0,none,NaN,none
6,3,1930,263190,3.73,0,0,0,0,none,NaN,none
7,3,2000,263190,3.73,0,0,0,0,none,NaN,none
8,3,2610,263190,3.73,0,0,0,0,none,NaN,none
9,3,4110,263190,3.73,0,0,0,0,none,NaN,none


## 3. Are children more frequently in the presence of adults compared to other children?


In [23]:

query_03 = """
WITH RandomSubjects AS (
    SELECT DISTINCT s.child_id, s.video_name, v.video_id, s.age_at_recording
    FROM Subjects s
    JOIN Videos v ON s.video_name = v.video_path
    ORDER BY RANDOM()
    LIMIT 40
),
-- Ziehe alle möglichen Frames aus den Videos der RandomSubjects
AllFrames AS (
    SELECT 
        rs.child_id, 
        rs.video_id, 
        rs.age_at_recording,
        f.frame_number
    FROM RandomSubjects rs
    JOIN (
        -- Hier alle möglichen Frames pro Video bestimmen
        SELECT video_id, frame_number
        FROM Detections_with_speaker
        GROUP BY video_id, frame_number
    ) f ON rs.video_id = f.video_id
),
-- Ziehe zufällig 5000 Frames
RandomFrames AS (
    SELECT *
    FROM AllFrames
    ORDER BY RANDOM()
    LIMIT 5000
)
SELECT 
    rf.video_id,
    rf.frame_number,
    rf.child_id,
    rf.age_at_recording AS age,
    COALESCE(CASE WHEN CAST(d.object_class AS INTEGER) = 0 OR CAST(d.object_class AS INTEGER) = 2 THEN 1 ELSE 0 END, 0) AS child_present,
    COALESCE(CASE WHEN CAST(d.object_class AS INTEGER) = 1 OR CAST(d.object_class AS INTEGER) = 3 THEN 1 ELSE 0 END, 0) AS adult_present
FROM RandomFrames rf
LEFT JOIN Detections_with_speaker d
    ON rf.video_id = d.video_id
   AND rf.frame_number = d.frame_number
WHERE d.object_class IN ('0', '1', '2', '3')  -- Only include child and adult classes
ORDER BY rf.child_id, rf.video_id, rf.frame_number;
"""

def merge_duplicates_03(df):
    grouped = df.groupby(['video_id', 'frame_number', 'child_id', 'age'])
    merged_rows = []

    for name, group in grouped:
        child_present = group['child_present'].max()
        adult_present = group['adult_present'].max()

        merged_row = {
            'video_id': name[0],
            'frame_number': name[1],
            'child_id': name[2],
            'age': name[3],
            'child_present': child_present,
            'adult_present': adult_present,
        }

        # generate person_age_class from child_present and adult_present
        if child_present == 1 and adult_present == 1:
            merged_row['person_age_class'] = 'both'
        elif child_present == 1:
            merged_row['person_age_class'] = 'child'
        elif adult_present == 1:
            merged_row['person_age_class'] = 'adult'
        else:
            merged_row['person_age_class'] = 'none'

        merged_rows.append(merged_row)

    return pd.DataFrame(merged_rows)


In [26]:
# Load data
with sqlite3.connect('/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db') as conn:
    person_presence_03_df = pd.read_sql_query(query_03, conn)

person_presence_03_df_merged = merge_duplicates_03(person_presence_03_df)
person_presence_03_df_merged.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/person_presence_03_df.csv', index=False)

len(person_presence_03_df_merged), person_presence_03_df_merged.head(10)

(2727,
    video_id  frame_number  child_id   age  child_present  adult_present  \
 0         4            40    257511  3.84              0              1   
 1         4          1400    257511  3.84              0              1   
 2         4          5330    257511  3.84              1              0   
 3         4          5340    257511  3.84              1              0   
 4         4          5930    257511  3.84              1              0   
 5         4          6050    257511  3.84              1              0   
 6         4          6180    257511  3.84              1              0   
 7         4          6290    257511  3.84              1              0   
 8         4          6510    257511  3.84              1              0   
 9         4          6680    257511  3.84              1              0   
 
   person_age_class  
 0            adult  
 1            adult  
 2            child  
 3            child  
 4            child  
 5            child  
 

## 4. How does the composition of social interactions change with age?


#### model <- interaction ~ age + (1|child_id)

In [ ]:

query_04 = """
WITH RandomSubjects AS (
    SELECT DISTINCT s.child_id, s.video_name, v.video_id, s.age_at_recording
    FROM Subjects s
    JOIN Videos v ON s.video_name = v.video_path
    ORDER BY RANDOM()
    LIMIT 40
),
-- Ziehe alle möglichen Frames aus den Videos der RandomSubjects
AllFrames AS (
    SELECT 
        rs.child_id, 
        rs.video_id, 
        rs.age_at_recording,
        f.frame_number
    FROM RandomSubjects rs
    JOIN (
        -- Hier alle möglichen Frames pro Video bestimmen
        SELECT video_id, frame_number
        FROM Detections_with_speaker
        GROUP BY video_id, frame_number
    ) f ON rs.video_id = f.video_id
),
-- Ziehe zufällig 5000 Frames
RandomFrames AS (
    SELECT *
    FROM AllFrames
    ORDER BY RANDOM()
    LIMIT 5000
)
SELECT 
    rf.video_id,
    rf.frame_number,
    rf.child_id,
    rf.age_at_recording AS age,
    COALESCE(CASE WHEN CAST(d.object_class AS INTEGER) = 0 OR CAST(d.object_class AS INTEGER) = 2 THEN 1 ELSE 0 END, 0) AS child_present,
    COALESCE(CASE WHEN CAST(d.object_class AS INTEGER) = 1 OR CAST(d.object_class AS INTEGER) = 3 THEN 1 ELSE 0 END, 0) AS adult_present
FROM RandomFrames rf
LEFT JOIN Detections_with_speaker d
    ON rf.video_id = d.video_id
   AND rf.frame_number = d.frame_number
WHERE d.object_class IN ('0', '1', '2', '3')  -- Only include child and adult classes
ORDER BY rf.child_id, rf.video_id, rf.frame_number;
"""

def merge_duplicates_04(df):
    grouped = df.groupby(['video_id', 'frame_number', 'child_id', 'age'])
    merged_rows = []

    for name, group in grouped:
        child_present = group['child_present'].max()
        adult_present = group['adult_present'].max()

        merged_row = {
            'video_id': name[0],
            'frame_number': name[1],
            'child_id': name[2],
            'age': name[3],
            'child_present': child_present,
            'adult_present': adult_present,
        }

        # generate person_age_class from child_present and adult_present
        if child_present == 1 and adult_present == 1:
            merged_row['person_age_class'] = 'both'
        elif child_present == 1:
            merged_row['person_age_class'] = 'child'
        elif adult_present == 1:
            merged_row['person_age_class'] = 'adult'
        else:
            merged_row['person_age_class'] = 'none'

        merged_rows.append(merged_row)

    return pd.DataFrame(merged_rows)


## 5. How does the frequency of toy use differ between solo and social play contexts?

In [ ]:
query = """
WITH RandomSubjects AS (
    -- Select 5 random subjects
    SELECT DISTINCT s.child_id, s.video_name, v.video_id, s.age_at_recording
    FROM Subjects s
    JOIN Videos v ON s.video_name = v.video_path
    ORDER BY RANDOM()
    LIMIT 10
),
RandomFrames AS (
    -- Select 500 random frames per subject
    SELECT DISTINCT d.frame_number, d.video_id
    FROM Detections d
    JOIN RandomSubjects rs ON d.video_id = rs.video_id
    GROUP BY d.frame_number, d.video_id
    ORDER BY RANDOM()
    LIMIT 500
),
SocialContext AS (
    SELECT 
        d.frame_number,
        d.video_id,
        CASE
            WHEN MAX(CASE WHEN d.object_class IN (1,3) THEN 1 ELSE 0 END) = 1 
            AND MAX(CASE WHEN d.object_class IN (0,2) THEN 1 ELSE 0 END) = 1 
            THEN 'child and adult present'
            WHEN MAX(CASE WHEN d.object_class IN (1,3) THEN 1 ELSE 0 END) = 1 
            THEN 'adult present'
            WHEN MAX(CASE WHEN d.object_class IN (0,2) THEN 1 ELSE 0 END) = 1 
            THEN 'child present'
            ELSE 'alone'
        END as social
    FROM Detections d
    JOIN RandomFrames rf ON d.frame_number = rf.frame_number AND d.video_id = rf.video_id
    GROUP BY d.frame_number, d.video_id
),
ObjectTypes AS (
    -- Create all possible object types
    SELECT 
        'book' as object_type, 5 as object_class UNION ALL
        SELECT 'toy', 6 UNION ALL
        SELECT 'kitchenware', 7 UNION ALL
        SELECT 'screen', 8 UNION ALL
        SELECT 'food', 9 UNION ALL
        SELECT 'other_object', 10
),
FrameObjects AS (
    -- Get unique object presence per frame and object type
    SELECT DISTINCT
        rf.frame_number,
        rf.video_id,
        ot.object_type,
        ot.object_class,
        MAX(CASE WHEN d.object_class IS NOT NULL THEN 1 ELSE 0 END) as object_present
    FROM RandomFrames rf
    CROSS JOIN ObjectTypes ot
    LEFT JOIN Detections d ON 
        rf.frame_number = d.frame_number 
        AND rf.video_id = d.video_id 
        AND ot.object_class = d.object_class
    GROUP BY rf.frame_number, rf.video_id, ot.object_type, ot.object_class
)

SELECT DISTINCT
    fo.video_id,
    fo.frame_number as frame_id,
    rs.child_id as ID,
    fo.object_present,
    fo.object_type,
    rs.age_at_recording as age,
    COALESCE(sc.social, 'alone') as social
FROM 
    FrameObjects fo
    JOIN RandomSubjects rs ON fo.video_id = rs.video_id
    LEFT JOIN SocialContext sc ON fo.frame_number = sc.frame_number AND fo.video_id = sc.video_id
ORDER BY 
    rs.child_id, fo.video_id, fo.frame_number, fo.object_type;
"""

# Load data
with sqlite3.connect('/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db') as conn:
    object_df = pd.read_sql_query(query, conn)

# Convert categorical variables to factors
object_df['social'] = pd.Categorical(object_df['social'])
object_df['object_type'] = pd.Categorical(object_df['object_type'])
object_df['ID'] = pd.Categorical(object_df['ID'])

print("Data shape:", object_df.shape)
print("\nNumber of unique subjects:", object_df['ID'].nunique())
print("\nVariable types:")
print(object_df.dtypes)
print("\nSample data (showing first 12 rows to see multiple object types per frame):")
display(object_df.head(12))

Data shape: (3000, 7)

Number of unique subjects: 10

Variable types:
video_id             int64
frame_id             int64
ID                category
object_present       int64
object_type       category
age                float64
social            category
dtype: object

Sample data (showing first 12 rows to see multiple object types per frame):


,video_id,frame_id,ID,object_present,object_type,age,social
0,151,330,257108,0,book,4.25,adult present
1,151,330,257108,0,food,4.25,adult present
2,151,330,257108,0,kitchenware,4.25,adult present
3,151,330,257108,0,other_object,4.25,adult present
4,151,330,257108,0,screen,4.25,adult present
5,151,330,257108,0,toy,4.25,adult present
6,151,1920,257108,0,book,4.25,child present
7,151,1920,257108,0,food,4.25,child present
8,151,1920,257108,0,kitchenware,4.25,child present
9,151,1920,257108,0,other_object,4.25,child present


In [5]:
# Convert columns to categorical type
object_df['object_type'] = object_df['object_type'].astype('category')
object_df['social_context'] = object_df['social'].astype('category')

In [7]:
# save df to csv
object_df.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/object_test_results.csv', index=False)

# Social Context (Alone Yes or No)

In [21]:
query = """
WITH RandomSubjects AS (
    SELECT DISTINCT s.child_id, s.video_name, v.video_id, s.age_at_recording
    FROM Subjects s
    JOIN Videos v ON s.video_name = v.video_path
    ORDER BY RANDOM()
    LIMIT 10
),
AllFrames AS (
    SELECT DISTINCT frame_number, video_id
    FROM Detections
    WHERE video_id IN (SELECT video_id FROM RandomSubjects)
),
RandomFrames AS (
    SELECT frame_number, video_id
    FROM AllFrames
    ORDER BY RANDOM()
    LIMIT 250
),
FaceInfo AS (
    SELECT 
        rf.video_id,
        rf.frame_number,
        rs.child_id,
        rs.age_at_recording,
        
        -- Binary flags
        CASE WHEN d.object_class = 0 THEN 1 ELSE 0 END AS child_person,
        CASE WHEN d.object_class = 1 THEN 1 ELSE 0 END AS adult_person,
        
        CASE WHEN d.object_class = 2 THEN 1 ELSE 0 END AS child_face,
        CASE WHEN d.object_class = 3 THEN 1 ELSE 0 END AS adult_face,
        
        -- Gaze
        CASE WHEN d.object_class = 2 THEN d.gaze_direction ELSE NULL END AS child_gaze,
        CASE WHEN d.object_class = 3 THEN d.gaze_direction ELSE NULL END AS adult_gaze,

        -- Proximity
        CASE WHEN d.object_class = 2 THEN d.proximity ELSE NULL END AS child_proximity,
        CASE WHEN d.object_class = 3 THEN d.proximity ELSE NULL END AS adult_proximity
    

    FROM RandomFrames rf
    JOIN RandomSubjects rs ON rf.video_id = rs.video_id
    LEFT JOIN Detections d 
        ON d.video_id = rf.video_id 
        AND d.frame_number = rf.frame_number 
        AND d.object_class IN (2, 3)  -- only faces

    GROUP BY rf.video_id, rf.frame_number, rs.child_id, rs.age_at_recording
)

SELECT 
    video_id,
    frame_number AS frame_id,
    child_id AS ID,
    age_at_recording AS age,
    child_person,
    adult_person,
    child_face,
    adult_face,
    child_gaze,
    adult_gaze,
    child_proximity,
    adult_proximity
FROM FaceInfo
ORDER BY child_id, video_id, frame_number;
"""

# Load data
with sqlite3.connect('/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db') as conn:
    social_df = pd.read_sql_query(query, conn)

print("Data shape:", social_df.shape)
print("\nNumber of unique subjects:", social_df['ID'].nunique())
print("\nVariable types:")
print(social_df.dtypes)
print("\nSample data (showing first 12 rows to see multiple object types per frame):")
display(social_df.head(12))

Data shape: (250, 12)

Number of unique subjects: 10

Variable types:
video_id             int64
frame_id             int64
ID                   int64
age                float64
child_person         int64
adult_person         int64
child_face           int64
adult_face           int64
child_gaze         float64
adult_gaze         float64
child_proximity    float64
adult_proximity    float64
dtype: object

Sample data (showing first 12 rows to see multiple object types per frame):


,video_id,frame_id,ID,age,child_person,adult_person,child_face,adult_face,child_gaze,adult_gaze,child_proximity,adult_proximity
0,173,2150,257578,4.04,0,0,0,1,NaN,0.0,NaN,0.658033
1,173,13170,257578,4.04,0,0,1,0,1.0,NaN,0.566699,NaN
2,173,13540,257578,4.04,0,0,1,0,1.0,NaN,0.570673,NaN
3,173,14080,257578,4.04,0,0,0,0,NaN,NaN,NaN,NaN
4,173,18730,257578,4.04,0,0,1,0,1.0,NaN,0.597570,NaN
5,173,19070,257578,4.04,0,0,1,0,1.0,NaN,0.621938,NaN
6,173,21910,257578,4.04,0,0,0,0,NaN,NaN,NaN,NaN
7,173,22380,257578,4.04,0,0,0,1,NaN,1.0,NaN,0.417363
8,173,22950,257578,4.04,0,0,0,0,NaN,NaN,NaN,NaN
9,173,28540,257578,4.04,0,0,0,0,NaN,NaN,NaN,NaN


In [ ]:
# create column person_present if at least one of adult or child is present
social_df['person_present'] = social_df['child_person'] | social_df['adult_person'] | social_df['child_face'] | social_df['adult_face']
social_df['child_present'] = social_df['child_person'] | social_df['child_face']
social_df['adult_present'] = social_df['adult_person'] | social_df['adult_face']

In [24]:
# save df to csv
social_df.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/social_test_results.csv', index=False)

In [33]:
import pandas as pd
import re
from datetime import datetime, time

# Load Excel file and prepare data
df = pd.read_excel("/home/nele_pauline_suffo/ProcessedData/quantex_data_sheet.xlsx") 

# remove rows with filename starting with "quantex_at_home_pakistan"
df = df[~df['file name  (generated automatically)'].str.startswith("quantex_at_home_pakistan")]
df = df[~df['file name  (generated automatically)'].str.startswith("quantex_at_home_id_yyyy")]

# Normalize annotation column
df['is_annotated'] = df['Annotated'].isin(['Yes', 'review', 'in progress'])

# Convert duration to minutes
def time_to_minutes(t):
    if isinstance(t, time):
        return t.hour * 60 + t.minute + (t.second / 60.0)
    elif isinstance(t, str):
        # Parse string format if needed
        try:
            t = datetime.strptime(t, '%H:%M:%S').time()
            return t.hour * 60 + t.minute + (t.second / 60.0)
        except:
            return 0
    return 0

# Add duration in minutes column
df['duration_minutes'] = df['DURATION'].apply(time_to_minutes)

# Get training set (already annotated videos)
train_df = df[df['is_annotated']].copy()
train_length = train_df['duration_minutes'].sum()

# Calculate target duration for val and test (10% each of final dataset)
# train_length is 80%, so divide by 0.8 to get total length, then take 10%
final_total_length = train_length / 0.8
target_length = final_total_length * 0.1

# Get candidate videos for val/test (not yet annotated)
candidates = df[~df['is_annotated']].copy()
candidates = candidates.sort_values('duration_minutes')

# Function to select videos closest to target duration
def select_videos_for_split(candidates_df: pd.DataFrame, target_duration: float, max_videos: int) -> pd.DataFrame:
    """Select videos aiming for target duration with unique IDs."""
    candidates_sorted = candidates_df.copy()  # Create a copy to avoid warnings
    selected = []
    current_duration = 0
    used_ids = set()
    
    while current_duration < target_duration and len(selected) < max_videos:
        # Get remaining candidates using loc
        mask = ~candidates_sorted['ID'].isin(used_ids)
        remaining = candidates_sorted.loc[mask].copy()
        
        if remaining.empty:
            break
            
        # Calculate gap to target using loc
        remaining.loc[:, 'gap_to_target'] = (
            target_duration - (current_duration + remaining['duration_minutes'])
        ).abs()
        
        best_match = remaining.nsmallest(1, 'gap_to_target').iloc[0]
        
        if current_duration + best_match['duration_minutes'] > target_duration * 1.1:
            break
            
        selected.append(best_match)
        used_ids.add(best_match['ID'])
        current_duration += best_match['duration_minutes']
        
        if len(selected) >= 10 and current_duration >= target_duration * 0.9:
            break
    
    return pd.DataFrame(selected)

# Get validation and test sets
candidates_mask = (candidates['duration_minutes'] < target_length * 0.3)  # Reduce max duration threshold
remaining_candidates = candidates.loc[candidates_mask].copy()

# Select validation set
val_df = select_videos_for_split(remaining_candidates, target_length, max_videos=12)

# Select test set from remaining videos
test_mask = (~candidates['ID'].isin(val_df['ID']) & 
            (candidates['duration_minutes'] < target_length * 0.3))
remaining_candidates = candidates.loc[test_mask].copy()
test_df = select_videos_for_split(remaining_candidates, target_length, max_videos=12)


# Verify unique IDs
train_ids = set(train_df['ID'])
val_ids = set(val_df['ID'])
test_ids = set(test_df['ID'])
# check for overlap in all three sets
overlap = train_ids.intersection(val_ids).union(train_ids.intersection(test_ids)).union(val_ids.intersection(test_ids))

if overlap:
    print("Warning: Found overlapping IDs between validation and test sets!")
    print(f"Overlapping IDs: {overlap}")
    
val_length = val_df['duration_minutes'].sum()
test_length = test_df['duration_minutes'].sum()
total_length = train_length + val_length + test_length

print("✅ Training set (already annotated):")
print(f"Number of videos: {len(train_df)}")

print("\n✅ Validation candidates:")
print(f"Number of videos: {len(val_df)}")
print(val_df[['file name  (generated automatically)', 'duration_minutes']])

print("\n✅ Test candidates:")
print(f"Number of videos: {len(test_df)}")
print(test_df[['file name  (generated automatically)', 'duration_minutes']])

print(f"\nDuration Summary:")
print(f"Training:    {train_length:.2f} minutes ({(train_length/total_length)*100:.1f}%)")
print(f"Validation:  {val_length:.2f} minutes ({(val_length/total_length)*100:.1f}%)")
print(f"Test:        {test_length:.2f} minutes ({(test_length/total_length)*100:.1f}%)")
print(f"Total:       {total_length:.2f} minutes")

print(f"\nTarget duration for val/test: {target_length:.2f} minutes")

Overlapping IDs: {258704.0}
✅ Training set (already annotated):
Number of videos: 75

✅ Validation candidates:
Number of videos: 8
       file name  (generated automatically)  duration_minutes
533  quantex_at_home_id266050_2024_09_15_01              30.0
588  quantex_at_home_id267079_2025_02_11_02              30.0
581  quantex_at_home_id266971_2024_09_06_02              30.0
159  quantex_at_home_id258704_2022_05_10_01              30.0
573  quantex_at_home_id266822_2022_11_14_01              30.0
141  quantex_at_home_id258541_2023_03_26_01              30.0
547  quantex_at_home_id266063_2024_09_19_01              30.0
340  quantex_at_home_id263194_2025_02_15_01               4.9

✅ Test candidates:
Number of videos: 8
       file name  (generated automatically)  duration_minutes
550  quantex_at_home_id266151_2024_09_14_01              30.0
554  quantex_at_home_id266352_2025_01_28_01              30.0
135  quantex_at_home_id258309_2023_03_12_01              30.0
564  quantex_at_home_id

In [ ]:
import pandas as pd
import re
from datetime import datetime, time

# Load Excel file and prepare data
df = pd.read_excel("/home/nele_pauline_suffo/ProcessedData/quantex_data_sheet.xlsx") 

# remove rows with filename starting with "quantex_at_home_pakistan"
df = df[~df['file name  (generated automatically)'].str.startswith("quantex_at_home_pakistan")]
df = df[~df['file name  (generated automatically)'].str.startswith("quantex_at_home_id_yyyy")]

# Normalize annotation column
df['is_annotated'] = df['Annotated'].isin(['Yes', 'review', 'in progress'])

# Convert duration to minutes
def time_to_minutes(t):
    if isinstance(t, time):
        return t.hour * 60 + t.minute + (t.second / 60.0)
    elif isinstance(t, str):
        # Parse string format if needed
        try:
            t = datetime.strptime(t, '%H:%M:%S').time()
            return t.hour * 60 + t.minute + (t.second / 60.0)
        except:
            return 0
    return 0

# Add duration in minutes column
df['duration_minutes'] = df['DURATION'].apply(time_to_minutes)

# Get training set (already annotated videos)
train_df = df[df['is_annotated']].copy()
train_length = train_df['duration_minutes'].sum()

# Calculate target duration for val and test (10% each of final dataset)
# train_length is 80%, so divide by 0.8 to get total length, then take 10%
final_total_length = train_length / 0.8
target_length = final_total_length * 0.1

# Get candidate videos for val/test (not yet annotated)
candidates = df[~df['is_annotated']].copy()
candidates = candidates.sort_values('duration_minutes')

# Function to select videos closest to target duration
def select_videos_for_split(candidates_df: pd.DataFrame, target_duration: float, max_videos: int) -> pd.DataFrame:
    """Select videos aiming for target duration with unique IDs."""
    candidates_sorted = candidates_df.copy()  # Create a copy to avoid warnings
    selected = []
    current_duration = 0
    used_ids = set()
    
    while current_duration < target_duration and len(selected) < max_videos:
        # Get remaining candidates using loc
        mask = ~candidates_sorted['ID'].isin(used_ids)
        remaining = candidates_sorted.loc[mask].copy()
        
        if remaining.empty:
            break
            
        # Calculate gap to target using loc
        remaining.loc[:, 'gap_to_target'] = (
            target_duration - (current_duration + remaining['duration_minutes'])
        ).abs()
        
        best_match = remaining.nsmallest(1, 'gap_to_target').iloc[0]
        
        if current_duration + best_match['duration_minutes'] > target_duration * 1.1:
            break
            
        selected.append(best_match)
        used_ids.add(best_match['ID'])
        current_duration += best_match['duration_minutes']
        
        if len(selected) >= 10 and current_duration >= target_duration * 0.9:
            break
    
    return pd.DataFrame(selected)

# Get validation and test sets
candidates_mask = (candidates['duration_minutes'] < target_length * 0.3)  # Reduce max duration threshold
remaining_candidates = candidates.loc[candidates_mask].copy()

# Select validation set
val_df = select_videos_for_split(remaining_candidates, target_length, max_videos=12)

# Select test set from remaining videos
test_mask = (~candidates['ID'].isin(val_df['ID']) & 
            (candidates['duration_minutes'] < target_length * 0.3))
remaining_candidates = candidates.loc[test_mask].copy()
test_df = select_videos_for_split(remaining_candidates, target_length, max_videos=12)


# Verify unique IDs
train_ids = set(train_df['ID'])
val_ids = set(val_df['ID'])
test_ids = set(test_df['ID'])
# check for overlap in all three sets
overlap = train_ids.intersection(val_ids).union(train_ids.intersection(test_ids)).union(val_ids.intersection(test_ids))

if overlap:
    print("Warning: Found overlapping IDs between validation and test sets!")
    print(f"Overlapping IDs: {overlap}")
    
val_length = val_df['duration_minutes'].sum()
test_length = test_df['duration_minutes'].sum()
total_length = train_length + val_length + test_length

print("✅ Training set (already annotated):")
print(f"Number of videos: {len(train_df)}")

print("\n✅ Validation candidates:")
print(f"Number of videos: {len(val_df)}")
print(val_df[['file name  (generated automatically)', 'duration_minutes']])

print("\n✅ Test candidates:")
print(f"Number of videos: {len(test_df)}")
print(test_df[['file name  (generated automatically)', 'duration_minutes']])

print(f"\nDuration Summary:")
print(f"Training:    {train_length:.2f} minutes ({(train_length/total_length)*100:.1f}%)")
print(f"Validation:  {val_length:.2f} minutes ({(val_length/total_length)*100:.1f}%)")
print(f"Test:        {test_length:.2f} minutes ({(test_length/total_length)*100:.1f}%)")
print(f"Total:       {total_length:.2f} minutes")

print(f"\nTarget duration for val/test: {target_length:.2f} minutes")

Overlapping IDs: {258704.0}
✅ Training set (already annotated):
Number of videos: 75

✅ Validation candidates:
Number of videos: 10
       file name  (generated automatically)  duration_minutes
710  quantex_at_home_id284216_2025_03_14_05         30.000000
446  quantex_at_home_id264585_2023_08_16_02         30.000000
436  quantex_at_home_id264514_2025_01_13_02         30.000000
197  quantex_at_home_id260478_2022_11_05_01         30.000000
440  quantex_at_home_id264556_2024_12_18_01         30.000000
191  quantex_at_home_id260178_2023_08_12_02         30.000000
407  quantex_at_home_id264089_2023_05_14_03         30.000000
340  quantex_at_home_id263194_2025_02_15_01          4.900000
107  quantex_at_home_id258239_2020_08_23_03          0.750000
459  quantex_at_home_id264666_2025_02_28_02          1.066667

✅ Test candidates:
Number of videos: 10
       file name  (generated automatically)  duration_minutes
589  quantex_at_home_id267079_2025_02_11_03         30.000000
581  quantex_at_home_